In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [2]:
def autoregressive_model(burnin, n, c, phi):
    p = len(phi)
    epsilon = np.random.normal(size=n + burnin)
    ar_values = np.zeros(n + burnin)

    for t in range(p, n + burnin):
        ar_values[t] = c + sum([phi[i] * ar_values[t - i - 1] for i in range(p)]) + epsilon[t]

    return ar_values[burnin:]

ar2_model = autoregressive_model(100, 5000, 8, [1.3, -0.7])
ar2_series = pd.Series(ar2_model)

In [3]:
models = []
for p in range(1, 5):
    model = ARIMA(ar2_series, order=(p, 0, 0))
    model_fit = model.fit()
    models.append((p, model_fit.aic))

aic_values = pd.DataFrame(models, columns=['p', 'AIC'])

best_model = aic_values.loc[aic_values['AIC'].idxmin()]

print(f"The best model is AR({best_model['p']}) with AIC = {best_model['AIC']}")

The best model is AR(2.0) with AIC = 14245.040255913227


The AIC score is calculated for models of order from 1 to 4, each model's AIC score is recorded in the DataFrame, the best model is the model with the lowest AIC score. In this case, the best model is AR(2) with an AIC score of 14245.

In [4]:
def moving_average(burnin, n, c, theta):
    q = len(theta)
    epsilon = np.random.normal(size=n + burnin)
    ma_values = np.zeros(n + burnin)

    for t in range(q, n + burnin):
        ma_values[t] = c + epsilon[t] + sum([theta[i] * epsilon[t - i - 1] for i in range(q)])

    return ma_values[burnin:]

ma_values_2 = moving_average(100, 5000, 0, [-1, 0.8])
ma2_series = pd.Series(ma_values_2)

In [5]:
models = []
for q in range(1, 5):
    model = ARIMA(ma2_series, order=(0, 0, q))
    model_fit = model.fit()
    models.append((q, model_fit.aic))

aic_values = pd.DataFrame(models, columns=['q', 'AIC'])

best_model = aic_values.loc[aic_values['AIC'].idxmin()]

print(f"The best model is MA({best_model['q']}) with AIC = {best_model['AIC']}")

The best model is MA(3.0) with AIC = 14273.43146336835


The process is repeated for the MA model, here the best model was found to be MA(3) with an AIC score of 14273.